In [ ]:
from train import *
dde.config.set_random_seed(1)

# Cantilever Beam

In [ ]:
layer_size = [1] + [30]*3 +[1]
activation = "swish"
initializer = "Glorot uniform"
data = data = dde.data.PDE(geom, ode, bcs_c, 15, 2 , solution=analy_cb, num_test=100)
net = dde.nn.FNN(layer_size, activation, initializer)
model, history = solver(data, net, lr=1e-3,iter=6000)

In [ ]:
x_values = np.linspace(0, L, 100).reshape(-1, 1)

y_pred = model.predict(x_values)
y_exact = analy_cb(x_values)

y_true = analy_frb_mspl(x_values.flatten())
l2_error = np.linalg.norm(y_pred.flatten() - y_true, 2) / np.linalg.norm(y_true, 2)

plt.figure(figsize=(10, 6))
plt.plot(x_values, y_exact, label="Exact Solution", color="blue")
plt.plot(x_values, y_pred, label="PINN Prediction", color="red", linestyle="--")
plt.xlabel("x")
plt.ylabel("u(x)")
plt.title(f"PINN vs Analytical: Deflection of Cantilever Beam Under Uniform Load, L2 Relative Error: {l2_error:.2e}")
plt.legend()
plt.grid()
plt.show

# Fully Restrained Beam

In [ ]:
layer_size = [1] + [50]*3 +[1]
activation = "swish"
initializer = "Glorot uniform"
data = data = dde.data.PDE(geom, ode, bcs_fr, 100, 50 , solution=analy_frb, num_test=100)
net = dde.nn.FNN(layer_size, activation, initializer)
model, history = solver(data, net, lr=1e-3,iter=28000)

In [ ]:
x_values = np.linspace(0, L, 100).reshape(-1, 1)

y_pred = model.predict(x_values)
w_analytical = analy_frb(x_values)

y_true = analy_frb_mspl(x_values.flatten())
l2_error = np.linalg.norm(y_pred.flatten() - y_true, 2) / np.linalg.norm(y_true, 2)

plt.figure(figsize=(10, 6))
plt.plot(x_values, y_pred, label="PINN Prediction", color="red", linestyle="--")
plt.plot(x_values, w_analytical, label="Exact Solution", color="blue")
plt.xlabel("x")
plt.ylabel("w(x)")
plt.title(f"PINN vs Analytical: Deflection of Fully Restrained Beam Under Uniform Load, L2 Relative Error: {l2_error:.2e}")
plt.legend()
plt.grid()
plt.show()

# Fully Restrained Beam with Mid-Span Point Load

In [ ]:
layer_size = [1] + [50]*3 +[1]
activation = "swish"
initializer = "Glorot uniform"
lw = [1e-14, 1, 1, 1 ,1]
data = data = dde.data.PDE(geom, ode_mspl, bcs_fr, 50, 20 , solution=analy_frb_mspl, num_test=100)
net = dde.nn.FNN(layer_size, activation, initializer)

filepath = "model_cb_mspl_frb/model.ckpt"
weight = CustomLossWeightCallback()
best_model = dde.callbacks.ModelCheckpoint(filepath, verbose=0, save_better_only=True, period=1, monitor='train loss')


model, history = solver(data, net, lr=1e-3,iter=60000, loss_weights=lw, callback=[weight, best_model])

In [ ]:
x_values = np.linspace(0, L, 100).reshape(-1, 1)

y_pred = model.predict(x_values)

y_true = analy_frb_mspl(x_values.flatten())
l2_error = np.linalg.norm(y_pred.flatten() - y_true, 2) / np.linalg.norm(y_true, 2)

plt.figure(figsize=(8, 5))
plt.plot(x_values, -y_pred, label="PINN prediction")
plt.plot(x_values, -y_true, "--", label="Analytical solution")
plt.xlabel("x")
plt.ylabel("Deflection w(x)")
plt.title(f"Fully Restrained Beam Under Mid-Span Point Load, L2 Relative Error: {l2_error:.2e}" )
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("frb_mspl.png", dpi=300)